<a href="https://colab.research.google.com/github/mounirboulwafa/AIOX-LABS-NLP-Case-Study/blob/master/aiox_labs_nlp_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FastAi Setup (Fast-BERT)

In [ ]:
pip install fast-bert

In [ ]:
from fastai.text import *

## Corpus Loading

In [ ]:
#get the text data

!git clone https://github.com/fbougares/TSAC.git

Cloning into 'TSAC'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 2), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [ ]:
path = '/content/TSAC'

In [ ]:
#load train data

df_train_neg = pd.read_csv(path + '/train_neg.txt' , sep='\n', names=['text'])
df_train_neg['label'] = 'neg'

df_train_pos = pd.read_csv(path + '/train_pos.txt' , sep='\n', names=['text'])
df_train_pos['label'] = 'pos'

In [ ]:
#load test data

df_test_neg = pd.read_csv(path + '/test_neg.txt' , sep='\n', names=['text'])
df_test_neg['label'] = 'neg'

df_test_pos = pd.read_csv(path + '/test_pos.txt' , sep='\n', names=['text'])
df_test_pos['label'] = 'pos'

In [ ]:
df_test_neg.head()

,text,label
0,تي ماللخر راك انسان بوهالي تعمل في Buzz و بن ع...,neg
1,Nn mouch 7louwa faza,neg
2,mabladkom 3bed tfouuuuh,neg
3,تفووووووه عليك و علا والديك على عايلتك و على ا...,neg
4,لا يليق بهذا البرنامج,neg


In [ ]:
df_valid = df_test_neg.append(df_test_pos, ignore_index=True, verify_integrity=True).reset_index()
df_train = df_train_neg.append(df_train_pos, ignore_index=True, verify_integrity=True).reset_index()


In [ ]:
df_train['label'].value_counts()

pos    4944
neg    4688
Name: label, dtype: int64

## Train Darija Language Model

In [ ]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, text_cols=['text'], label_cols=['label'])
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.2, pretrained=False)

In [ ]:
learn.fit(epochs=24, lr=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.651710,5.738576,0.213811,00:13
1,6.348965,5.740358,0.213811,00:13
2,6.245673,5.689252,0.213811,00:13
3,6.184901,5.601099,0.233426,00:13
4,6.113988,5.538193,0.233566,00:13
5,6.029366,5.451387,0.234208,00:13
6,5.922055,5.322144,0.255636,00:13
7,5.798790,5.187602,0.260631,00:13
8,5.685239,5.103485,0.264174,00:13
9,5.563542,5.007313,0.268443,00:13


In [ ]:
learn.save_encoder('trained-darija-language-model')

## Opining Mining (Classification)

In [ ]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, vocab=data_lm.train_ds.vocab, bs=64, text_cols=['text'], label_cols=['label'])

In [ ]:
classifier = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.2)

In [ ]:
classifier.load_encoder('trained-darija-language-model')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (9632 items)
x: TextList
xxbos xxmaj tafeh 3ale5er,xxbos xxmaj merig bsaraha,xxbos xxmaj mala kalb ! ! !,xxbos xxmaj rak xxmaj mast xxrep 10 h rak xxmaj last xxrep 5 h,xxbos ماهيش متربية و مادام تشجع في لواط هذي موش تونسية و الإسلام xxunk وحدة عاهرة
y: CategoryList
neg,neg,neg,neg,neg
Path: /content/TSAC;

Valid: LabelList (2302 items)
x: TextList
xxbos تي ماللخر راك انسان بوهالي تعمل في xxmaj buzz و بن علي ماهوش راجع xxunk ",xxbos xxmaj nn mouch 7louwa faza,xxbos xxunk 3bed tfo xxrep 4 u h,xxbos تف xxrep 6 و ه عليك و علا والديك على عايلتك و على اصلك و على xxunk و على الي xxunk و باش xxunk و على الي xxunk و xxunk و على xxunk و على الارض الي xxunk و على تونس الي xxunk تف xxrep 18 و ه,xxbos لا يليق بهذا البرنامج
y: CategoryList
neg,neg,neg,neg,neg
Path: /content/TSAC;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(7936, 400, padding_idx=1)
      (encoder_dp): EmbeddingDrop

In [ ]:
classifier.fit(epochs=24, lr=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.595022,0.578830,0.683319,00:16
1,0.572540,0.565815,0.685925,00:16
2,0.555134,0.526322,0.720678,00:15
3,0.545881,0.532508,0.704605,00:17
4,0.539872,0.515158,0.732841,00:16
5,0.543904,0.519002,0.721112,00:16
6,0.525213,0.524747,0.707211,00:16
7,0.519948,0.515211,0.720678,00:17
8,0.509084,0.507409,0.725891,00:17
9,0.515076,0.511169,0.724587,00:16


In [ ]:
def get_sentiment(text):
  predict = classifier.predict(text)[2]
  score_pos = predict.numpy()[1]
  score_neg = predict.numpy()[0]

  if score_pos < score_neg:
    print(text + ' :  Neg a ' + str(format(score_neg,'.2%')))
  else:
    print(text + ' :  Pos a ' + str(format(score_pos,'.2%')))
 

In [ ]:
get_sentiment('3jebni')
get_sentiment('3jbni')
get_sentiment('روعة ماشاء الله')
get_sentiment('bravo 3lik')
get_sentiment('برافو برافو برافو')

3jebni :  Pos a 89.96%
3jbni :  Pos a 92.70%
روعة ماشاء الله :  Pos a 96.13%
bravo 3lik :  Pos a 98.92%
برافو برافو برافو :  Pos a 99.92%
